In [7]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
from pprint import pprint

# .env を同じフォルダから読み込む（パス指定は不要）
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# モデル名
MODEL_NAME = "gpt-4o-mini"


In [8]:
import os

api_key = os.getenv("OPENAI_API_KEY")
print("APIキーが読み込めているか？ →", api_key is not None)


APIキーが読み込めているか？ → True


In [9]:
# 検索結果を返す関数の作成
def get_search_result(keyword, when="1", unit="m"):
    result = {
        "result": [
            {"title": "1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました"},
            {"title": "1929年（昭和4年）八重洲口が開設されました"},
            {"title": "2014年（平成26年）東京駅開業100周年を迎えました"},
        ]
    }

    # ★ OpenAI SDK v2 では dict のまま返す。json.dumps() は不要。
    return result


In [10]:
# ツール定義（SDK v2 用）
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_search_result",
            "description": "指定したキーワードの検索結果を取得する",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "検索するキーワード"
                    },
                    "when": {
                        "type": "number",
                        "description": "期間指定（日数など）"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["d", "h", "y"],
                        "description": "期間の単位（d=日、h=時間、y=年）"
                    }
                },
                "required": ["keyword"]
            }
        }
    }
]


In [11]:
# 言語モデルが直接回答できる質問
question = "日本の首都はどこにありますか？"

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",  # ツールを使うかどうかはモデルに任せる
)

# レスポンスの表示
from pprint import pprint
pprint(response.model_dump())


{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'annotations': [],
                          'audio': None,
                          'content': '日本の首都は東京都にあります。東京は日本の政治、経済、文化の中心地であり、多くの重要な機関や企業が集まっています。',
                          'function_call': None,
                          'refusal': None,
                          'role': 'assistant',
                          'tool_calls': None}}],
 'created': 1765149666,
 'id': 'chatcmpl-CkILqbaaqEcTd2tIN0NszFBKuNTFM',
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_a460d7e2b7',
 'usage': {'completion_tokens': 39,
           'completion_tokens_details': {'accepted_prediction_tokens': 0,
                                         'audio_tokens': 0,
                                         'reasoning_tokens': 0,
                                         'rejected_prediction_tokens': 0},
           'pro

In [12]:
# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "user", "content": question},
    ],
    tools=tools,
    tool_choice="auto",  # 必要ならツールを呼ぶ、不要なら通常回答
)

from pprint import pprint

# 全体構造を見たい場合
pprint(response.model_dump())

# モデルがツールを呼ぼうとしているか確認
print("tool_calls:", response.choices[0].message.tool_calls)

# 直接の回答（ツールを使わなかった場合に入る）
print("content:", response.choices[0].message.content)


{'choices': [{'finish_reason': 'tool_calls',
              'index': 0,
              'logprobs': None,
              'message': {'annotations': [],
                          'audio': None,
                          'content': None,
                          'function_call': None,
                          'refusal': None,
                          'role': 'assistant',
                          'tool_calls': [{'function': {'arguments': '{"keyword":"東京駅 '
                                                                    'イベント","when":30,"unit":"d"}',
                                                       'name': 'get_search_result'},
                                          'id': 'call_a97rBV4w0a829wWmWlHUixLL',
                                          'type': 'function'}]}}],
 'created': 1765149876,
 'id': 'chatcmpl-CkIPElojlZhulRey4tj4yqzQQZfUz',
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'service_tier': 'default',
 'system_fingerprint': 'fp_a460d7e2b7',
 'u

In [13]:
import json
from pprint import pprint

# モデルがツール呼出と判断した
tool_calls = response.choices[0].message.tool_calls or []

if response.choices[0].finish_reason == "tool_calls" and tool_calls:
    # 言語モデルの回答からツールを取得
    tool = tool_calls[0]

    # 関数名の取得
    function_name = tool.function.name
    print(f"関数名：{function_name}")

    # 引数の取得（JSON文字列 → dict）
    arguments = json.loads(tool.function.arguments)
    print(f"引数：{arguments}")

    # 関数の実行（get_search_result など）
    function_response = globals()[function_name](**arguments)
    print("関数の実行結果：")
    pprint(function_response)  # dict をそのまま表示

    # ツールの実行結果を JSON 文字列にして model に渡す
    tool_content = json.dumps(function_response, ensure_ascii=False)

    # 関数の実行結果を messages に加えて言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": tool_content,
            },
        ],
    )

    # 言語モデルからの回答を出力
    print("言語モデルからの回答：")
    print(response_after_tool_call.choices[0].message.content.strip())

else:
    # 関数呼び出しでなければ、単に言語モデルからの回答を出力
    print("言語モデルからの回答：")
    print(response.choices[0].message.content.strip())


関数名：get_search_result
引数：{'keyword': '東京駅 イベント', 'when': 30, 'unit': 'd'}
関数の実行結果：
{'result': [{'title': '1914年（大正3年）中央停車場改め「東京駅」として営業を開始しました'},
            {'title': '1929年（昭和4年）八重洲口が開設されました'},
            {'title': '2014年（平成26年）東京駅開業100周年を迎えました'}]}
言語モデルからの回答：
最近1ヶ月以内の東京駅に関するイベントの検索結果としては、特定のイベント情報は見つかりませんでしたが、歴史的な情報や記念日についての内容がいくつかありました。具体的には、東京駅が1914年に開業したことや、1929年に八重洲口が開設されたこと、そして2014年に開業100周年を迎えたことが挙げられます。

最新のイベント情報については、公式ウェブサイトや地域のイベントカレンダーを確認することをお勧めします。
